# Python で気軽に化学・化学工学
# 第 9 章　モデルを適用できる x の範囲・領域を設定したり、推定結果の信頼性を議論する

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

わからないことがありましたら、関係する単語やエラーの文章などでウェブ検索してご自身で調べてみましょう。

### 沸点のデータセット(descriptors_8_with_boiling_point.csv)
Hall and Story が収集した[沸点のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci960375x)。294 個の化合物について、沸点 (Boiling Point) が測定されており、8 つの変数 (記述子) で化学構造が数値化されています。記述子は、分子量 (MolWt)、水素原子以外の原子で計算された分子量 (HeavyAtomMolWt)、価電子の数 (NumValenceElectrons)、水素原子以外の原子の数 (HeavyAtomCount)、窒素原子と酸素原子の数 (NOCount)、水素原子と炭素原子以外の原子の数 (NumHeteroatoms)、回転可能な結合の数 (NumRotatableBonds)、環の数 (RingCount) です。

データセットの読み込み

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_8_with_boiling_point.csv', index_col=0) # 沸点のデータセットの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 記述子を説明変数 x とします

In [ ]:
y = dataset.iloc[:, 0] # 沸点を目的変数 y とします

データセットのトレーニングデータとテストデータへの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# トレーニングデータとテストデータとに分割。shuffle=False として、上から順に(沸点が低い傾向のある化合物のみ)トレーニングデータを選択し、残りをテストデータとします
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=244, shuffle=False)

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの説明変数の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化

AD を設定せずに、PLS 解析をします。

In [ ]:
from sklearn import model_selection #クロスバリデーションにより推定値を計算するために使用
from sklearn.cross_decomposition import PLSRegression # PLS モデル構築やモデルを用いた y の値の推定に使用
from sklearn import metrics # r2 等の指標の計算に使用

In [ ]:
max_number_of_components = 6 # 最大の成分数

In [ ]:
fold_number = 10 # クロスバリデーションのfold数

In [ ]:
components = [] # 空の list の変数を作成して、成分数をこの変数に追加していきます同じく成分数をこの変数に追加していきます
r2_in_cv_all = [] # 空の list の変数を作成して、成分数ごとのクロスバリデーション後の r2 をこの変数に追加していきます

成分数を 1, 2, ..., max_number_of_components-1, max_number_of_components と変えて、それぞれクロスバリデーションにより推定値を計算

In [ ]:
for component in range(1, max_number_of_components+1):
    model = PLSRegression(n_components=component) # PLS モデルの宣言
    estimated_y_in_cv = pd.DataFrame(model_selection.cross_val_predict(model, autoscaled_x_train, autoscaled_y_train, cv=fold_number)) # クロスバリデーション推定値の計算し、DataFrame型に変換
    estimated_y_in_cv = estimated_y_in_cv * y_train.std() + y_train.mean() # スケールをもとに戻す
    r2_in_cv = metrics.r2_score(y_train, estimated_y_in_cv) # r2 を計算
    print(component, r2_in_cv) # 成分数と r2 を表示
    r2_in_cv_all.append(r2_in_cv)  # r2 を追加
    components.append(component) # 成分数を追加

In [ ]:
r2_in_cv_all # 念のため確認

成分数ごとのクロスバリデーション後の r<sup>2</sup> を図で確認

In [ ]:
import matplotlib.pyplot as plt # 図の描画に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(components, r2_in_cv_all) # 散布図
plt.xlabel('number of components') # x 軸の名前
plt.ylabel('cross-validated r2') # y 軸の名前
plt.show() # 以上の設定で描画

In [ ]:
optimal_component_number = components[r2_in_cv_all.index(max(r2_in_cv_all))] # 最適成分数を optimal_component_number に代入

In [ ]:
optimal_component_number # 念のため確認

最適成分数で PLS モデル構築

In [ ]:
model = PLSRegression(n_components=optimal_component_number)

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # 回帰モデルの構築

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # トレーニングデータの y の値を推定し、Pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # テストデータの y の値を推定して Pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

テストデータには推定誤差の大きなサンプルが存在します。これらは AD 外と考えられます。

### データ密度による AD の設定

k-NN によって計算されるデータ密度で AD を設定します。トレーニングデータにおける最も近い k 個のサンプルとの距離の平均が小さければ、AD 内と考えられます。

In [ ]:
from sklearn.neighbors import NearestNeighbors # k-NN

In [ ]:
k_in_knn = 3

k-NN モデルを表す変数の作成の仕方は第 6 回や第 7 回におけるクラス分類・回帰分析の k-NN と同じです。

In [ ]:
ad_model = NearestNeighbors(n_neighbors=k_in_knn, metric='euclidean') # k-NN モデルを表す変数の作成

`metric` の種類 (第五回のクラスタリングのときの `metric` とは異なることに注意してください)
- 'euclidean' : ユークリッド距離
- 'manhattan' : マンハッタン距離(シティブロック距離)

など。その他の距離については [こちら](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html) をご覧ください 

In [ ]:
ad_model.fit(autoscaled_x_train) # k-NN では、トレーニングデータの数値データを model_ad に格納することに対応します

次のセルでは、サンプルごとの k 最近傍サンプルとの距離に加えて、k 最近傍サンプルのインデックス番号も一緒に出力されるため、出力用の変数を 2 つにしています。またトレーニングデータでは k 最近傍サンプルの中に自分も含まれ、自分との距離の 0 を除いた距離を考える必要があるため、k_in_knn + 1 個と設定し直しています。

In [ ]:
knn_distance_between_autoscaled_x_train, knn_index_autoscaled_x_train = ad_model.kneighbors(autoscaled_x_train, n_neighbors=k_in_knn+1)

In [ ]:
knn_distance_between_autoscaled_x_train # サンプルごとの k最近傍サンプルとの距離

In [ ]:
knn_index_autoscaled_x_train # サンプルごとの k最近傍サンプルのインデックス番号

In [ ]:
knn_distance_between_autoscaled_x_train = pd.DataFrame(knn_distance_between_autoscaled_x_train) # DataFrame型に変換

In [ ]:
knn_distance_between_autoscaled_x_train.index = x_train.index # サンプル名をトレーニングデータのサンプル名にします

In [ ]:
knn_distance_between_autoscaled_x_train # 念のため確認

各サンプルについて、右になるにつれて距離が大きくなることを確認しましょう。すべてのサンプルにおいて、一番左は自分との距離になるため 0.0 になっています。

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train = knn_distance_between_autoscaled_x_train.iloc[:, 1:].mean(axis=1) # 自分以外の k_in_knn 個の距離の平均

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train # 念のため確認

次のセルで距離の平均の小さい順に並び替えます。なお、ascending=False とすると、大きい順に並び替えることができます。

In [ ]:
sorted_mean_of_knn_distance_between_autoscaled_x_train = mean_of_knn_distance_between_autoscaled_x_train.sort_values(ascending=True)

In [ ]:
sorted_mean_of_knn_distance_between_autoscaled_x_train # 念のため確認

In [ ]:
alpha = 0.8

In [ ]:
round(autoscaled_x_train.shape[0] * alpha) # 距離の小さい α ×100 % のサンプル数

In [ ]:
ad_threshold = sorted_mean_of_knn_distance_between_autoscaled_x_train.iloc[round(autoscaled_x_train.shape[0] * alpha) - 1] # トレーニング化合物の alpha % が含まれるようにしきい値を設定

In [ ]:
ad_threshold

#### テストデータに対して、AD の内か外かを判定

In [ ]:
knn_distance_between_autoscaled_x_train_test, knn_index_autoscaled_x_test = ad_model.kneighbors(autoscaled_x_test)

In [ ]:
knn_distance_between_autoscaled_x_train_test = pd.DataFrame(knn_distance_between_autoscaled_x_train_test) # DataFrame型に変換

In [ ]:
knn_distance_between_autoscaled_x_train_test.index = x_test.index # サンプル名をテストデータのサンプル名にします

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train_test = knn_distance_between_autoscaled_x_train_test.mean(axis=1) # k_in_knn 個の距離の平均

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train_test # 念のため確認

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train_test <= ad_threshold # モデルの適用範囲内のサンプル True

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train_test > ad_threshold # モデルの適用範囲外のサンプルは True

In [ ]:
y_test_inside_ad = y_test[mean_of_knn_distance_between_autoscaled_x_train_test <= ad_threshold] # AD 内のサンプルにおける、y の実測値

In [ ]:
len(y_test_inside_ad) # AD 内のサンプル数を確認

In [ ]:
estimated_y_test_inside_ad = estimated_y_test[mean_of_knn_distance_between_autoscaled_x_train_test <= ad_threshold] # AD 内のサンプルにおける、y の推定

In [ ]:
estimated_y_test_inside_ad.to_csv('estimated_y_test_inside_ad.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
y_test_outside_ad = y_test[mean_of_knn_distance_between_autoscaled_x_train_test > ad_threshold] # AD 外のサンプルにおける、y の実測値

In [ ]:
len(y_test_outside_ad) # AD 外のサンプル数を確認

In [ ]:
estimated_y_test_outside_ad = estimated_y_test[mean_of_knn_distance_between_autoscaled_x_train_test > ad_threshold]  # AD 外のサンプルにおける、y の推定値

In [ ]:
estimated_y_test_outside_ad.to_csv('estimated_y_test_outside_ad.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

AD 内のテストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test_inside_ad, estimated_y_test_inside_ad.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test_inside_ad.max(), estimated_y_test_inside_ad.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test_inside_ad.min(), estimated_y_test_inside_ad.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

AD 外のテストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test_inside_ad, estimated_y_test_inside_ad) # r2

In [ ]:
metrics.mean_absolute_error(y_test_inside_ad, estimated_y_test_inside_ad) # MAE

AD 外のテストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test_outside_ad, estimated_y_test_outside_ad.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test_outside_ad.max(), estimated_y_test_outside_ad.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test_outside_ad.min(), estimated_y_test_outside_ad.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

AD 外のテストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test_outside_ad, estimated_y_test_outside_ad) # r2

In [ ]:
metrics.mean_absolute_error(y_test_outside_ad, estimated_y_test_outside_ad) # MAE

AD 外のサンプルの MAE と比較して、AD 内のサンプルの MAE が小さくなっています。推定誤差が大きくなる可能性のあるサンプルを、検討対象から外すことができました。

### アンサンブル学習による AD の設定

今回は説明変数をランダムに選択して複数のモデルを作成します。`descriptors_8_with_boiling_point.csv` は説明変数の数が 8 と少ないため、17 に増やしたデータセット `descriptors_33_with_boiling_point.csv` を使用します。記述子の種類については[こちら](https://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-descriptors)をご覧ください。

In [ ]:
dataset = pd.read_csv('descriptors_33_with_boiling_point.csv', index_col=0) # 沸点のデータセットの読み込み

In [ ]:
dataset.shape

まずは先ほどと同様にして、AD を設定せずに PLS 解析を行います。

In [ ]:
x = dataset.iloc[:, 1:] # 記述子を説明変数 x とします

In [ ]:
y = dataset.iloc[:, 0] # 沸点を目的変数 y とします

データセットのトレーニングデータとテストデータへの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# トレーニングデータとテストデータとに分割。shuffle=False として、上から順に(沸点が低い傾向のある化合物のみ)トレーニングデータを選択し、残りをテストデータとします
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=244, shuffle=False)

In [ ]:
x_train.shape

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの説明変数の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化

PLS

In [ ]:
from sklearn import model_selection #クロスバリデーションにより推定値を計算するために使用
from sklearn.cross_decomposition import PLSRegression # PLS モデル構築やモデルを用いた y の値の推定に使用
from sklearn import metrics # r2 等の指標の計算に使用

In [ ]:
max_number_of_components = 15 # 最大の成分数

In [ ]:
fold_number = 10 # クロスバリデーションのfold数

In [ ]:
components = [] # 空の list の変数を作成して、成分数をこの変数に追加していきます同じく成分数をこの変数に追加していきます
r2_in_cv_all = [] # 空の list の変数を作成して、成分数ごとのクロスバリデーション後の r2 をこの変数に追加していきます

成分数を 1, 2, ..., max_number_of_components-1, max_number_of_components と変えて、それぞれクロスバリデーションにより推定値を計算

In [ ]:
for component in range(1, max_number_of_components+1):
    model = PLSRegression(n_components=component) # PLS モデルの宣言
    estimated_y_in_cv = pd.DataFrame(model_selection.cross_val_predict(model, autoscaled_x_train, autoscaled_y_train, cv=fold_number)) # クロスバリデーション推定値の計算し、DataFrame型に変換
    estimated_y_in_cv = estimated_y_in_cv * y_train.std() + y_train.mean() # スケールをもとに戻す
    r2_in_cv = metrics.r2_score(y_train, estimated_y_in_cv) # r2 を計算
    print(component, r2_in_cv) # 成分数と r2 を表示
    r2_in_cv_all.append(r2_in_cv)  # r2 を追加
    components.append(component) # 成分数を追加

In [ ]:
r2_in_cv_all # 念のため確認

成分数ごとのクロスバリデーション後の r<sup>2</sup> を図で確認

In [ ]:
import matplotlib.pyplot as plt # 図の描画に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(components, r2_in_cv_all) # 散布図
plt.xlabel('number of components') # x 軸の名前
plt.ylabel('cross-validated r2') # y 軸の名前
plt.show() # 以上の設定で描画

In [ ]:
optimal_component_number = components[r2_in_cv_all.index(max(r2_in_cv_all))] # 最適成分数を optimal_component_number に代入

In [ ]:
optimal_component_number # 念のため確認

最適成分数で PLS モデル構築

In [ ]:
model = PLSRegression(n_components=optimal_component_number)

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # 回帰モデルの構築

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # トレーニングデータの y の値を推定し、Pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # テストデータの y の値を推定して Pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

テストデータには推定誤差の大きなサンプルが存在します。これらは AD 外と考えられます。

#### アンサンブル学習による推定値の標準偏差の計算

今回は、アンサンブル学習における一つのモデルを構築するための説明変数として、元の説明変数の 8 割くらいを用いることにします。

In [ ]:
rate_of_selected_variables = 0.8

0 から 1 までの間に一様に分布する乱数を説明変数の数だけ生成して、その乱数値が rate_of_selected_variables を下回った説明変数を選択します。一様乱数を生成するために numpy というライブラリを用います。

## numpy
- ベクトル・行列計算のためのライブラリ
- 基礎統計量などの計算も可能
- for文などによる繰り返し処理は実行時間がかかるため、行列計算でうまく処理すると大幅な効率改善につながる
- [NumPy - Quickstart tutorial](https://docs.scipy.org/doc/numpy/user/quickstart.html)

In [ ]:
# 一般的には as np として省略した名前で import します
import numpy as np

In [ ]:
np.random.rand(10) # 10 個の一様乱数の生成。0 から 1 の間にちらばった値が生成されることを確認してください。実行するごとに異なる乱数が生成されます。

説明変数の数だけ乱数を発生させ、それが rate_of_selected_variables を下回る説明変数だけ選択します。

In [ ]:
random_x_variables = np.random.rand(x_train.shape[1])

In [ ]:
random_x_variables # 念のため確認

In [ ]:
random_x_variables < rate_of_selected_variables # rate_of_selected_variables を下回る要素だけ True

In [ ]:
selected_autoscaled_x_train = autoscaled_x_train.iloc[:, random_x_variables < rate_of_selected_variables] # True の説明変数のデータセットのみ選択

In [ ]:
selected_autoscaled_x_train # 念のため確認

アンサンブル学習におけるモデルの数を 100 とします。

In [ ]:
number_of_models = 100 # モデルの数
max_number_of_components = 10 # 最大の成分数
fold_number = 10 # クロスバリデーションのfold数

以下の内容について、モデルの数だけ for 文で回します。

1. 説明変数の選択
2. クロスバリデーションによるハイパーパラメータの最適化
3. モデル構築
4. トレーニングデータの y の推定
5. テストデータの y の推定

In [ ]:
from sklearn import model_selection #クロスバリデーションにより推定値を計算するために使用
from sklearn.cross_decomposition import PLSRegression # PLS モデル構築やモデルを用いた y の値の推定に使用
from sklearn import metrics # r2 等の指標の計算に使用

下のセルの計算が終了するには数分かかることもあります。ただ進捗状況を表示することで、どの程度まで計算が終了しているか確認できます。また、計算中はセルの左の領域が In[*]: となっており、計算が終了すると [] 内に数字が表示されます。

In [ ]:
estimated_y_train_all = pd.DataFrame() # 空の DataFrame 型を作成し、ここにモデルごとのトレーニングデータの y の推定結果を追加します
estimated_y_test_all = pd.DataFrame() # 空の DataFrame 型を作成し、ここにモデルごとのテストデータの y の推定結果を追加します
for model_number in range(number_of_models):
    print(model_number+1, '/', number_of_models) # 進捗状況の表示
    # 1. 説明変数の選択
    random_x_variables = np.random.rand(x_train.shape[1])
    selected_autoscaled_x_train = autoscaled_x_train.iloc[:, random_x_variables < rate_of_selected_variables] # True の説明変数のデータセットのみ選択
    selected_autoscaled_x_test = autoscaled_x_test.iloc[:, random_x_variables < rate_of_selected_variables] # True の説明変数のデータセットのみ選択
    # 2. クロスバリデーションによるハイパーパラメータの最適化
    r2_in_cv_all = [] # 空の list の変数を作成して、成分数ごとのクロスバリデーション後の r2 をこの変数に追加していきます
    for component in range(1, max_number_of_components+1):
        model = PLSRegression(n_components=component) # PLS モデルの宣言
        estimated_y_in_cv = pd.DataFrame(model_selection.cross_val_predict(model, selected_autoscaled_x_train, autoscaled_y_train, cv=fold_number)) # クロスバリデーション推定値の計算し、DataFrame型に変換
        estimated_y_in_cv = estimated_y_in_cv * y_train.std() + y_train.mean() # スケールをもとに戻す
        r2_in_cv = metrics.r2_score(y_train, estimated_y_in_cv) # r2 を計算
        r2_in_cv_all.append(r2_in_cv)  # r2 を追加
    optimal_component_number = components[r2_in_cv_all.index(max(r2_in_cv_all))] # 最適成分数を optimal_component_number に代入
    # 3. モデル構築
    model = PLSRegression(n_components=optimal_component_number)
    model.fit(selected_autoscaled_x_train, autoscaled_y_train) # 回帰モデルの構築
    # 4. トレーニングデータの y の推定
    estimated_y_train = pd.DataFrame(model.predict(selected_autoscaled_x_train)) # トレーニングデータの y の値を推定し、Pandas の DataFrame 型に変換
    estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します
    estimated_y_train_all = pd.concat([estimated_y_train_all, estimated_y_train], axis=1)
    # 5. テストデータの y の推定
    estimated_y_test = pd.DataFrame(model.predict(selected_autoscaled_x_test)) # テストデータの y の値を推定して Pandas の DataFrame 型に変換
    estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します
    estimated_y_test_all = pd.concat([estimated_y_test_all, estimated_y_test], axis=1)

In [ ]:
estimated_y_test_all # 念のため確認

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = estimated_y_train_all.mean(axis=1) # モデルごとの y の推定結果の平均値を、最終的な y の推定結果とします

In [ ]:
estimated_y_train = pd.DataFrame(estimated_y_train) # DataFrame型に変換

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = estimated_y_test_all.mean(axis=1) # モデルごとの y の推定結果の平均値を、最終的な y の推定結果とします

In [ ]:
estimated_y_test = pd.DataFrame(estimated_y_test) # DataFrame 型に変換

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

テストデータにおける推定値の標準偏差

In [ ]:
std_of_estimated_y_test = estimated_y_test_all.std(axis=1)

In [ ]:
std_of_estimated_y_test = pd.DataFrame(std_of_estimated_y_test) # DataFrame 型に変換

In [ ]:
std_of_estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
std_of_estimated_y_test.columns = ['std_of_estimated_y'] # 列名を変更

In [ ]:
std_of_estimated_y_test.to_csv('std_of_estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータにおける推定値の標準偏差と推定誤差の関係

In [ ]:
y_test_abs_error = abs(y_test - estimated_y_test.iloc[:, 0]) # テストデータの y の誤差の絶対値

In [ ]:
y_test_abs_error

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(std_of_estimated_y_test.iloc[:, 0], y_test_abs_error)  # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
plt.xlabel("std of estimated y") # x 軸の名前
plt.ylabel("estimation y-error") # y 軸の名前
plt.show() # 以上の設定で描画

推定値の標準偏差が大きいほど推定誤差の絶対値が大きくなる可能性があることがわかります。

#### 練習問題
k-NN を用いて構築したクラス分類モデルを対象にして、データ密度により AD を設定して、AD 内の推定精度と AD 外の推定精度を比較しましょう。あやめのデータセットを用いてもよいですし、お持ちのデータセットを用いても構いません。あやめのデータセットを用いたコード例は下にあります。

#### コード例 (あやめのデータセットを用いた場合)
k-NN を用いて構築したクラス分類モデルを対象にして、データ密度により AD を設定して、AD 内の推定精度と AD 外の推定精度を比較しましょう。あやめのデータセットを用いてもよいですし、お持ちのデータセットを用いても構いません。

あやめのデータの読み込み

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('iris_with_species.csv', index_col=0) # あやめのデータの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 説明変数 x

In [ ]:
y = dataset.iloc[:, 0] # 目的変数 y

データセットのトレーニングデータとテストデータへの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=120, shuffle=True, random_state=3)

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの説明変数の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

AD を設定せずに、k-NN 解析をします。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # k-NN で使用
from sklearn import model_selection #クロスバリデーションにより推定値を計算するために使用
from sklearn import metrics # r2 等の指標の計算に使用

In [ ]:
max_number_of_k = 10 # 最大の k の数

In [ ]:
fold_number = 5 # クロスバリデーションのfold数

k を 1, 2, ..., max_number_of_k-1, max_number_of_k と変えて、それぞれクロスバリデーションにより推定

In [ ]:
accuracy_in_cv_all = [] # 空の list の変数を作成して、成分数ごとのクロスバリデーション後の 正解率 をこの変数に追加していきます
ks = [] # 同じく k の値をこの変数に追加していきます
for k in range(1, max_number_of_k+1):
    model = KNeighborsClassifier(n_neighbors=k, metric='euclidean') # k-NN モデルの宣言
    estimated_y_in_cv = pd.DataFrame(model_selection.cross_val_predict(model, autoscaled_x_train, y_train, cv=fold_number)) # クロスバリデーション推定値の計算し、DataFrame型に変換
    accuracy_in_cv = metrics.accuracy_score(y_train, estimated_y_in_cv) # 正解率を計算
    print(k, accuracy_in_cv) # k の値と r2 を表示
    accuracy_in_cv_all.append(accuracy_in_cv)  # r2 を追加
    ks.append(k) # k の値を追加

In [ ]:
accuracy_in_cv_all # 念のため確認

k の値ごとのクロスバリデーション後の正解率を図で確認

In [ ]:
import matplotlib.pyplot as plt # 図の描画に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.scatter(ks, accuracy_in_cv_all) # 散布図
plt.xlabel('k') # x 軸の名前
plt.ylabel('cross-validated accuracy') # y 軸の名前
plt.show() # 以上の設定で描画

In [ ]:
max(accuracy_in_cv_all) # クロスバリデーション後の r2 の最大値

In [ ]:
accuracy_in_cv_all.index(max(accuracy_in_cv_all)) # クロスバリデーション後の正解率が最大値となる index 番号 (同じ値が複数あるときは、もっとも index 番号の小さい番号のみ出力されます)

In [ ]:
optimal_k = ks[accuracy_in_cv_all.index(max(accuracy_in_cv_all))] # 最適成分数を optimal_k に代入

In [ ]:
optimal_k # 念のため確認

In [ ]:
model = KNeighborsClassifier(n_neighbors=optimal_k, metric='euclidean')

In [ ]:
model.fit(autoscaled_x_train, y_train) # クラス分類モデルの構築。k-NN では、トレーニングデータの数値データとクラスを model に格納することに対応します

トレーニングデータのクラスの推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # Pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_class'] # 列名を変更

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの混同行列

In [ ]:
from sklearn import metrics

In [ ]:
set(y_train) # クラスの種類

In [ ]:
class_types = list(set(y_train)) # リスト型に変換。これで混同行列における縦と横のクラスの順番を定めます

In [ ]:
class_types.sort() # アルファベット順に並び替え

In [ ]:
class_types # 念のため確認

In [ ]:
metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types) # 混同行列。array 型で与えられます

In [ ]:
confusion_matrix_train = pd.DataFrame(metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types)) # Pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_train.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_train.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_train # 念のため確認

In [ ]:
confusion_matrix_train.to_csv('confusion_matrix_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_train, estimated_y_train) # 正解率

テストデータのクラスの推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
model.predict(autoscaled_x_test) # テストデータのクラスを推定。array 型で出力されます

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # Pandas の DataFrame 型に変換

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_class'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの混同行列

In [ ]:
metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types) # 混同行列。array 型で与えられます

In [ ]:
confusion_matrix_test = pd.DataFrame(metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types)) # Pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test # 念のため確認

In [ ]:
confusion_matrix_test.to_csv('confusion_matrix_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test, estimated_y_test) # 正解率

### データ密度による AD の設定

k-NN によって計算されるデータ密度で AD を設定します。トレーニングデータにおける最も近い k 個のサンプルとの距離の平均が小さければ、AD 内と考えられます。

In [ ]:
from sklearn.neighbors import NearestNeighbors # k-NN

In [ ]:
k_in_knn = 3

k-NN モデルを表す変数の作成の仕方は第六回や第七回におけるクラス分類・回帰分析の k-NN と同じです。

In [ ]:
ad_model = NearestNeighbors(n_neighbors=k_in_knn, metric='euclidean') # k-NN モデルを表す変数の作成

`metric` の種類 (第五回のクラスタリングのときの `metric` とは異なることに注意してください)
- 'euclidean' : ユークリッド距離
- 'manhattan' : マンハッタン距離(シティブロック距離)

など。その他の距離については [こちら](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html) をご覧ください 

In [ ]:
ad_model.fit(autoscaled_x_train) # k-NN では、トレーニングデータの数値データを model_ad に格納することに対応します

次のセルでは、サンプルごとの k 最近傍サンプルとの距離に加えて、k 最近傍サンプルのインデックス番号も一緒に出力されるため、出力用の変数を 2 つにしています。またトレーニングデータでは k 最近傍サンプルの中に自分も含まれ、自分との距離の 0 を除いた距離を考える必要があるため、k_in_knn + 1 個と設定し直しています。

In [ ]:
knn_distance_between_autoscaled_x_train, knn_index_autoscaled_x_train = ad_model.kneighbors(autoscaled_x_train, n_neighbors=k_in_knn+1)

In [ ]:
knn_distance_between_autoscaled_x_train # サンプルごとの k最近傍サンプルとの距離

In [ ]:
knn_index_autoscaled_x_train # サンプルごとの k最近傍サンプルのインデックス番号

In [ ]:
knn_distance_between_autoscaled_x_train = pd.DataFrame(knn_distance_between_autoscaled_x_train) # DataFrame型に変換

In [ ]:
knn_distance_between_autoscaled_x_train.index = x_train.index # サンプル名をトレーニングデータのサンプル名にします

In [ ]:
knn_distance_between_autoscaled_x_train # 念のため確認

各サンプルについて、右になるにつれて距離が大きくなることを確認しましょう。すべてのサンプルにおいて、一番左は自分との距離になるため 0.0 になっています。

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train = knn_distance_between_autoscaled_x_train.iloc[:, 1:].mean(axis=1) # 自分以外の k_in_knn 個の距離の平均

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train # 念のため確認

In [ ]:
sorted_mean_of_knn_distance_between_autoscaled_x_train = mean_of_knn_distance_between_autoscaled_x_train.sort_values(ascending=True) # 距離の平均の小さい順に並び替え

In [ ]:
sorted_mean_of_knn_distance_between_autoscaled_x_train # 念のため確認

In [ ]:
alpha = 0.6

In [ ]:
round(autoscaled_x_train.shape[0] * alpha) # 距離の小さい α ×100 % のサンプル数

In [ ]:
ad_threshold = sorted_mean_of_knn_distance_between_autoscaled_x_train.iloc[round(autoscaled_x_train.shape[0] * alpha) - 1] # トレーニング化合物の alpha % が含まれるようにしきい値を設定

In [ ]:
ad_threshold

#### テストデータに対して、AD の内か外かを判定

In [ ]:
knn_distance_between_autoscaled_x_train_test, knn_index_autoscaled_x_test = ad_model.kneighbors(autoscaled_x_test)

In [ ]:
knn_distance_between_autoscaled_x_train_test = pd.DataFrame(knn_distance_between_autoscaled_x_train_test) # DataFrame型に変換

In [ ]:
knn_distance_between_autoscaled_x_train_test.index = x_test.index # サンプル名をテストデータのサンプル名にします

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train_test = knn_distance_between_autoscaled_x_train_test.mean(axis=1) # k_in_knn 個の距離の平均

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train_test # 念のため確認

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train_test <= ad_threshold # モデルの適用範囲内のサンプル True

In [ ]:
mean_of_knn_distance_between_autoscaled_x_train_test > ad_threshold # モデルの適用範囲外のサンプルは True

In [ ]:
y_test_inside_ad = y_test[mean_of_knn_distance_between_autoscaled_x_train_test <= ad_threshold] # AD 内のサンプルにおける、y の実測値

In [ ]:
len(y_test_inside_ad) # AD 内のサンプル数を確認

In [ ]:
estimated_y_test_inside_ad = estimated_y_test[mean_of_knn_distance_between_autoscaled_x_train_test <= ad_threshold] # AD 内のサンプルにおける、y の推定

In [ ]:
estimated_y_test_inside_ad.to_csv('estimated_y_test_inside_ad.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
y_test_outside_ad = y_test[mean_of_knn_distance_between_autoscaled_x_train_test > ad_threshold] # AD 外のサンプルにおける、y の実測値

In [ ]:
len(y_test_outside_ad) # AD 外のサンプル数を確認

In [ ]:
estimated_y_test_outside_ad = estimated_y_test[mean_of_knn_distance_between_autoscaled_x_train_test > ad_threshold]  # AD 外のサンプルにおける、y の推定値

In [ ]:
estimated_y_test_outside_ad.to_csv('estimated_y_test_outside_ad.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

AD 内のテストデータの混同行列

In [ ]:
metrics.confusion_matrix(y_test_inside_ad, estimated_y_test_inside_ad, labels=class_types) # 混同行列。array 型で与えられます

In [ ]:
confusion_matrix_test_inside_ad = pd.DataFrame(metrics.confusion_matrix(y_test_inside_ad, estimated_y_test_inside_ad, labels=class_types)) # Pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test_inside_ad.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test_inside_ad.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test_inside_ad # 念のため確認

In [ ]:
confusion_matrix_test_inside_ad.to_csv('confusion_matrix_test_inside_ad.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test_inside_ad, estimated_y_test_inside_ad) # 正解率

AD 外のテストデータの混同行列

In [ ]:
metrics.confusion_matrix(y_test_outside_ad, estimated_y_test_outside_ad, labels=class_types) # 混同行列。array 型で与えられます

In [ ]:
confusion_matrix_test_outside_ad = pd.DataFrame(metrics.confusion_matrix(y_test_outside_ad, estimated_y_test_outside_ad, labels=class_types)) # Pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test_outside_ad.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test_outside_ad.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test_outside_ad # 念のため確認

In [ ]:
confusion_matrix_test_outside_ad.to_csv('confusion_matrix_test_outside_ad.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test_outside_ad, estimated_y_test_outside_ad) # 正解率

AD 内の正解率と比較して、AD 外の正解率が低いことがわかります。